In [1]:
data<-read.csv("Forces3.csv")

In [2]:
#Check for categorical data. The 3th and 96th columns are categorical
data[1:5,1:5]

predictor,predictor.1,predictor.2,predictor.3,predictor.4
3.0990676,30.31381,174.9512,8,301.3000
1.6074460,17.83223,212.2065,99,375.8420
4.8223026,39.69040,199.9276,15,151.1736
0.6624255,65.08108,175.8600,8,242.8898
1.5118722,78.65205,131.7224,9,250.1637


In [3]:
#Graphing the data, it looks like they are linear or quadratic, with large standard deviation
# jpeg("all_data.jpg", width=900, height=30000, res=120)
# par(mfrow=c(96,3),mar = rep(2, 4))
# for (i in (1:3)){
#     for (j in (1:96)){
#         plot(data[,j],data[,96+i])
#     }
# }
# dev.off()

In [4]:
#Shuffle rows before separation into x and y
data<- data[sample(nrow(data)),]

In [5]:
#Seperate
X_data=data[-c(97:99)]
y_data=data[c(97:99)]

In [6]:
#A little bit of stack overflow borrowing, the model.moatrix 
#line returns on hot encoded columns for things that are factors
ohe_cols<-function(x){
     xf=factor(x)
     return(model.matrix(~xf+0))
}

In [7]:
#One hot encodes specified columns of data, using above function
ohe_matrix<-function(X,cols){
    X<-cbind(X,lapply(X[cols], ohe_cols))
    X<-X[-cols]
    return(X)
}

In [8]:
#Apply oh encoder to the data 
cols=c(4,96)
X_data<-ohe_matrix(X_data, cols)

In [9]:
X_train<-X_data[1:(nrow(X_data)/100*70),]
y_train<-y_data[1:(nrow(y_data)/100*70),]

In [10]:
nrow(y_data)/100*85-nrow(X_data)/100*70

[1] 19401

In [11]:
X_val<-X_data[(nrow(X_data)/100*70):(nrow(X_data)/100*85),]
y_val<-y_data[(nrow(y_data)/100*70):(nrow(X_data)/100*85),]
X_test<-X_data[(nrow(X_data)/100*85):(nrow(X_data)),]
y_test<-y_data[(nrow(y_data)/100*85):(nrow(X_data)),]

In [12]:
#Clear some unused variables
rm(X_data)
rm(y_data)

In [13]:
#Calculate values for normalization
# X_tr_mean<-lapply(X_train,mean)
# X_tr_std<-lapply(X_train, sd)
# y_tr_mean<-lapply(y_train,mean)
# y_tr_std<-lapply(y_train, sd)

In [14]:
#Normalize X
# X_tr_norm<-(X_train-X_tr_mean)/X_tr_std
# X_val_norm<-(X_val-X_tr_mean)/X_tr_std
# X_test_norm<-(X_test-X_tr_mean)/X_tr_std

In [15]:
#Normalize y
# y_tr_norm<-(y_train-y_tr_mean)/y_tr_std
# y_val_norm<-(y_val-y_tr_mean)/y_tr_std
# y_test_norm<-(y_test-y_tr_mean)/y_tr_std

In [16]:
norm<-function(X,X_tr){
    for(i in 1:ncol(X)){
        X[,i]<-(X[,i] - min(X_tr[,i]))/(
        max(X_tr[,i]) - min(X_tr[,i]))
    }
    return(X)
}

In [17]:
X_tr_norm<-norm(X_train,X_train)
X_val_norm<-norm(X_val,X_train)
X_test_norm<-norm(X_test,X_train)

y_tr_norm<-norm(y_train,y_train)
y_val_norm<-norm(y_val,y_train)
y_test_norm<-norm(y_test,y_train)

In [155]:
#Make phi
phi_transform<-function(X){
    phi<-cbind(bias=rep(1,nrow(X)),X,X**2)
    return(phi)
}

In [19]:
#Clean out unused variables again. Probably not necessary, but the data set is reasonably large
rm(X_train)
rm(y_train)
rm(X_val)
rm(y_val)
rm(X_test)
rm(y_test)

In [20]:
X_tr_norm<-as.matrix(X_tr_norm)
X_val_norm<-as.matrix(X_val_norm)
X_test_norm<-as.matrix(X_test_norm)
y_tr_norm<-as.matrix(y_tr_norm)
y_val_norm<-as.matrix(y_val_norm)
y_test_norm<-as.matrix(y_test_norm)

In [156]:
#Create phis for test train validate
phi_tr<-phi_transform(X_tr_norm)
phi_val<-phi_transform(X_val_norm)
phi_test<-phi_transform(X_test_norm)

In [166]:
errbest<-10000000

In [203]:
w<-matrix(rnorm(ncol(y_tr_norm)*ncol(phi_tr)),ncol(phi_tr),ncol(y_tr_norm))
w_best<-w

In [204]:
#Set the number of epochs
epochs<-2000

In [205]:
#Initialize w and set hyper parameters
error<-c()
epoch<-c()
eta<-0.0000005
lam1<-.0001
lam2<-.001
for (i in 1:epochs){
    w<-w_best
    for (j in 1:50){
        #calculate error on validation set
        y_hat_val<-phi_val%*%w
        error<-c(error,sum(diag(t(y_val_norm-y_hat_val)%*%(y_val_norm-y_hat_val))))
        epoch<-c(epoch,50*(i-1)+j)
        if (error[(i-1)*50+j]<errbest){
            errbest<-error[(i-1)*50+j]
    #        print("Best Error Updated")
            w_best<-w
            }
        #Calculate new w using training set
        y_hat_tr<-phi_tr%*%w
        w<-w-eta*(t(phi_tr)%*%(y_hat_tr-y_tr_norm))-lam1*sign(w)-lam2*w
    }
    if (errbest!=error[(i-1)*50+j]){
        eta<-eta/5
    }
}

In [206]:
errbest

[1] 12359.92

In [207]:
(i-1)*50+j

[1] 1e+05

In [202]:
length(error)

[1] 100000

In [190]:
errbest

[1] 12359.92

In [144]:
write.csv(w_best,"XX^2w_best.csv")

In [120]:
w<-as.matrix(read.csv("x2wreg", header = TRUE))[,2:4]
w<-apply(w, 2, as.numeric)

In [124]:
nrow(w)

[1] 217

In [125]:
w<-w[c(1:95,102,96,99,104,98,103,100,101,105,97,106,109,108,107,110:203,210,204,207,212,206,211,208,209,213,205,214,217,216,215),]

In [126]:
y_hat_val<-phi_val%*%w
sum(diag(t(y_val_norm-y_hat_val)%*%(y_val_norm-y_hat_val)))

[1] 1.572893

In [104]:
sum(diag(t(w_best-w)%*%(w_best-w)))

[1] 122.0877

In [114]:
w

bias,0.300302459,-0.133149848,0.28729438
predictor,-0.081139017,-0.099713004,0.03502481
predictor.1,-0.054257500,0.032412832,0.04073031
predictor.2,-0.036053909,0.017001319,-0.09858752
predictor.4,0.042133692,-0.115627749,-0.17756457
predictor.5,-0.180639420,0.022149719,-0.08884081
predictor.6,0.124487240,0.061663044,-0.12985277
predictor.7,-0.032616703,0.026939266,0.05804812
predictor.8,0.073514777,-0.080724115,0.10389279
predictor.9,0.030370601,0.147903818,0.11934110
predictor.10,0.118784363,0.051369188,-0.10070211
